# Modelo Score de Sentimientos

## Preprocesado

In [1]:
!pip install demoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 1.5 MB/s eta 0:00:00


In [18]:
import demoji
import re

def cleaning_text(text):
# TRATAR TRADUCCIONES EN REVIEWS, BORRA EL STRING "(ORIGINAL)" Y TODO A SU DERECHA
  keyword = "(Original)"
  keyword_index = text.find(keyword) # Encontrar la posición de la palabra clave
  if keyword_index != -1:
      text = text[:keyword_index + len(keyword)] # Eliminar todo después de la palabra clave

  text = text.replace("(Original)", "")
  text = text.replace("(Translated by Google)", "")

# TRATAR EMOJIS
  text = demoji.replace_with_desc(text, ' ') # reemplaza los emojis con palabras mas un espacio entre si

# BORRA SALTOS DE LINEA
  text = text.replace("\n", " ")
# BORRA CARACTERES NO ALFANUMERICOS
  text = re.sub(r'[^\w\s]', '', text)
# BORRA ESPACIOS AL PRINCIPIO Y AL FINAL Y CONVERTIR A MINUSCULAS
  text = text.strip()
  text = text.lower()
# DESPUES DE LIMPIADO VERIFICA SI HAY CARACTERES O NO, SI NO HAY LO CONVIERTE EN NULO
  if not text:
      return None
  return text

In [63]:
ejemplo = """(Translated by Google) Solo trip in May. Very attentive employees. I Can't complain. Clean, comfortable, big rooms! Air conditioning and excellent safe. Very nice and clean bathroom. Very good location.
Full breakfast, water dispenser and coffee all day.
I recommend it in price and quality! 😀💚

(Original)
Viaje sola en Mayo. Empleados muy atentos. Habitaciones limpias, cómodas, grandes! Aire acondicionado y caja fuerte excelente. Baño muy lindo y limpio. Muy buena ubicación.
Desayuno completo, dispenser de agua y cafe todo el día.
Lo recomiendo en precio y calidad!"""

In [64]:
cleaning_text(ejemplo)

'solo trip in may very attentive employees i cant complain clean comfortable big rooms air conditioning and excellent safe very nice and clean bathroom very good location full breakfast water dispenser and coffee all day i recommend it in price and quality  grinning face  green heart'

### Ejemplo de uso

In [25]:
import pandas as pd

data = {'reviews': [
    "I love this hotel",
    "👍",
    "Amazing experience!",
    "...-()"
]}
df = pd.DataFrame(data)

In [26]:
# Usando apply
df["cleaned"] = df["reviews"].apply(cleaning_text)

In [ ]:
# Usando comprehension lists (mas eficiente)
df["cleaned"] = [cleaning_text(x) for x in df["reviews"]]

In [27]:
df["cleaned"]

0     i love this hotel
1             thumbs up
2    amazing experience
3                  None
Name: cleaned, dtype: object

## Modelo

In [32]:
import nltk
nltk.download('vader_lexicon') # DESCARGAR DICCIONARIO UNA VEZ

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [55]:
from nltk.sentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()

def sentiment(text):
  text = cleaning_text(text) # LIMPIA EL TEXTO
  if not text: # SI EL TEXTO ESTA VACIO DESPUES DE LA LIMPIEZA, EL SCORE ES NEUTRO
    return 0
  else:   # SI NO ESTA VACIO, VADER LO CALCULA
    scores = analyzer.polarity_scores(text)
  return scores["compound"]

### Ejemplo de uso

In [56]:
import pandas as pd

data = {'reviews': [
    "I love this hotel",
    "👍",
    "Hated the food",
    "Amazing experience!",
    "...-()"
]}


df = pd.DataFrame(data)

In [57]:
# Usando comprehension lists (mas eficiente)
df["sentiment_score"] = [sentiment(x) for x in df["reviews"]]

In [58]:
df

,reviews,sentiment_score
0,I love this hotel,0.6369
1,👍,0.0000
2,Hated the food,-0.6369
3,Amazing experience!,0.5859
4,...-(),0.0000


# Para el EDA e Insights

Para el modelo no se incluyó la remoción de stopwords y lemmatización porque bajaban la precisión, pero puede ser util para la wordclouds en el EDA

## Stopwords

In [67]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords') # DESCARGAR UNA VEZ
nltk.download('punkt')

stopwords_nltk = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [82]:
from nltk import word_tokenize

def remove_stopwords(text):
  tokens = word_tokenize(text) # TOKENIZA EL TEXTO EN PALABRAS SINGULARES
  filtered_sentence = [] # CREA UNA LISTA VACIA
  for i in tokens:
    if i not in stopwords_nltk: # SI LA PALABRA NO ESTA EN LA LISTA DE STOPWORDS, LA GUARDA EN LA LISTA VACIA
        filtered_sentence.append(i)

  return " ".join(filtered_sentence) # JUNTA LAS PALABRAS DE LA LISTA

In [77]:
remove_stopwords(ejemplo)

"( Translated Google ) Solo trip May . Very attentive employees . I Ca n't complain . Clean , comfortable , big rooms ! Air conditioning excellent safe . Very nice clean bathroom . Very good location . Full breakfast , water dispenser coffee day . I recommend price quality ! 😀💚 ( Original ) Viaje sola en Mayo . Empleados muy atentos . Habitaciones limpias , cómodas , grandes ! Aire acondicionado caja fuerte excelente . Baño muy lindo limpio . Muy buena ubicación . Desayuno completo , dispenser de agua cafe todo el día . Lo recomiendo en precio calidad !"

## Lemamtizacion

In [72]:
import spacy

nlp = spacy.load('en_core_web_sm') # DESCARGAR MODELO PREENTRENADO UNA VEZ

In [78]:
def lemmatize(text):
  doc = nlp(text)
  tokens = []
  for token in doc:
      tokens.append(token)
  lemmatization = [token.lemma_ for token in doc]
  sentence = " ".join(lemmatization)
  return sentence

In [79]:
lemmatize(ejemplo)

'( translate by Google ) solo trip in May . very attentive employee . I can not complain . clean , comfortable , big room ! air conditioning and excellent safe . very nice and clean bathroom . very good location . \n full breakfast , water dispenser and coffee all day . \n I recommend it in price and quality ! 😀 💚 \n\n ( Original ) \n Viaje sola en Mayo . empleado muy atento . Habitaciones limpias , cómodas , grande ! aire acondicionado y caja fuerte excelente . Baño muy lindo y limpio . muy buena ubicación . \n Desayuno completo , dispenser de agua y cafe todo el día . \n Lo recomiendo en precio y calidad !'

## Juntando las funciones

In [83]:
def cleaning_stopwords_lemmatization(text):
  text = cleaning_text(text)
  text = remove_stopwords(text)
  text = lemmatize(text)
  return text

In [84]:
cleaning_stopwords_lemmatization(ejemplo)

'solo trip may attentive employee can not complain clean comfortable big room air condition excellent safe nice clean bathroom good location full breakfast water dispenser coffee day recommend price quality grin face green heart'